# Oppgave 7

In [11]:
import sqlite3
import pandas as pd
from tabulate import tabulate
import os

def get_file_name_of_newest_database():
    base_name = "database"
    counter = 1

    # Increment counter until a new file name is found
    while os.path.exists(f"{base_name}{counter}.db"):
        counter += 1
    counter -= 1
    return f"{base_name}{counter}.db"

name = get_file_name_of_newest_database()
conn = sqlite3.connect(name)
cursor = conn.cursor()

In [12]:
def hent_akter(skuespiller):
    query = """
    SELECT DISTINCT RPA.Actnummer, S1.TeaterStykkeNavn
    FROM Skuespiller S1
    JOIN RollePaAkt RPA ON S1.TeaterStykkeNavn = RPA.teaterstykkeNavn AND S1.RolleNavn = RPA.rolleNavn
    JOIN Ansatt A1 on S1.Identifikator = A1.ansattId
    WHERE A1.navn = ?
    """
    cursor.execute(query, (skuespiller,))
    akter = cursor.fetchall()
    return akter


def hent_skuespillere_i_samme_akt(skuespiller):
    akter = hent_akter(skuespiller)
    print(f"Skuespilleren {skuespiller} har vært med i følgende akter:")
    print(tabulate
          (akter, headers= ["Akt", "Teaterstykke"], tablefmt="grid"))
    query = """
    SELECT
        S1.teaterstykkenavn,
        S2.navn as Actor,
        S2.RolleNavn,
        A1.navn as CoActor,
        S1.rolleNavn,
        RPA.Actnummer
    FROM
        Skuespiller S1
        join Ansatt A1 on S1.Identifikator = A1.ansattId
        JOIN RollePaAkt RPA ON S1.TeaterStykkeNavn = RPA.teaterstykkeNavn AND S1.RolleNavn = RPA.rolleNavn
        JOIN (
            SELECT S2.TeaterStykkeNavn, S2.RolleNavn, A2.navn, RPA2.Actnummer, A2.AnsattId
            FROM Skuespiller S2
            JOIN Ansatt A2 on S2.Identifikator = A2.ansattId
            JOIN RollePaAkt RPA2 ON S2.TeaterStykkeNavn = RPA2.teaterstykkeNavn AND S2.RolleNavn = RPA2.rolleNavn
            WHERE A2.navn = ?
        ) as S2
        ON S1.TeaterStykkeNavn = S2.TeaterStykkeNavn AND RPA.Actnummer = S2.Actnummer And A1.AnsattId != S2.AnsattId
        order by S1.rolleNavn

    """
    
    cursor.execute(query, (skuespiller,))
    
    skuespillere = cursor.fetchall()
    return skuespillere

# get all actors by joining ansatt and skuespiller
query = """
SELECT DISTINCT A1.navn
FROM Skuespiller S1
JOIN Ansatt A1 on S1.Identifikator = A1.ansattId
"""
cursor.execute(query)
actors = cursor.fetchall()
actors_string = "\n".join([f"{i+1}. {actor[0]}" for i, actor in enumerate(actors)])
selected_actor = input(f'Velg en skuespiller \n {actors_string}')
if selected_actor.isdigit() and int(selected_actor) <= len(actors) and int(selected_actor) > 0:
    selected_actor = actors[int(selected_actor)-1][0]
else:
    print("Invalid input")
    exit()

skuespillere = hent_skuespillere_i_samme_akt(selected_actor)

headers = [description[0] for description in cursor.description]
print(f"Skuespilleren {selected_actor} har spilt sammen med følgende skuespillere:")
print(tabulate(skuespillere, headers=headers, tablefmt="grid"),)

Skuespilleren Emma Caroline Deichmann har vært med i følgende akter:
+-------+----------------+
|   Akt | Teaterstykke   |
+=======+================+
|     1 | Kongsemnene    |
+-------+----------------+
|     2 | Kongsemnene    |
+-------+----------------+
|     5 | Kongsemnene    |
+-------+----------------+
|     4 | Kongsemnene    |
+-------+----------------+
Skuespilleren Emma Caroline Deichmann har spilt sammen med følgende skuespillere:
+--------------------+-------------------------+-------------+----------------------------+-------------------+-------------+
| TeaterStykkeNavn   | Actor                   | RolleNavn   | CoActor                    | RolleNavn         |   Actnummer |
+====================+=========================+=============+============================+===================+=============+
| Kongsemnene        | Emma Caroline Deichmann | Sigrid      | Thomas Jensen Takyi        | Biskop Nikolas    |           1 |
+--------------------+-------------------------+